In [1]:
from gym import Env, spaces
import unittest
import numpy as np
import random

import argparse
import json
import os

import collections

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from stable_baselines.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines import PPO2

import gym
from gym.spaces import Discrete, Box, Dict
from gym.envs.registration import EnvSpec, register

import os
import os.path

import ray
from ray import tune
from ray.rllib.agents.dqn.dqn_policy_graph import *
from ray.rllib.agents.ppo.ppo_policy_graph import *
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.env import MultiAgentEnv
from ray.rllib.models.preprocessors import DictFlatteningPreprocessor, Preprocessor

from ray.tune import run_experiments
from ray.tune.registry import register_env

/anaconda3/envs/marl_routing/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [9]:
filename = "/Users/mtgibson/learning_wardrop/graph_results.txt"

filename2 = "/Users/mtgibson/learning_wardrop/model_results.txt"

In [10]:
class BraessEnv(MultiAgentEnv):
    """Traffic Environment that uses the Braess's network. 
       See https://github.com/openai/gym/blob/master/gym/core.py for more details.
    """
    
    def __init__(self, params=None):
        # Make the Braess's network
        self.network = BraessNetwork()
        self.routes = self.network.routes
        
        # Observation space contains each route and travel times
        self.num_routes = len(self.routes)
        self.observation_space = spaces.Box(low=0,high=float('+inf'),shape=(3,),dtype=np.float32)
        
        # Action space contains each route and flow distribution of population (decimal)
        self.action_space = action_spaces = spaces.Box(low=0,high=1,shape=(3,),dtype=np.float32)
        
        self.reward_range = (-float('inf'), 0)
        
        #Storage bins for data
        self.__path_flows = []
        self.__travel_times = []
        self.__avg_times = []
        return

    def step(self, action_dict):
        """Run one timestep of the environment's dynamics. 
        
        Env Step Procedure:
            (1) takes in routing distribution - comes from the action, 
            (2) calculate travel times for each path given flow on each path, 
            (3) return the reward which is the negative of the travel time.
            
        Note: We will give a reward to each path and will return an array/dictionary 
              as the reward for the population.
            
        Args:
            action (dictionary): A dictionary where the key is a population and values are another dictionary where
                                 the key is a path and the value is a flow distribution.
                                 We assume there is only 1 agent and thus one o-d pair
        
        Returns:
            next_observation (array): the travel times determined for each path
            reward (float): -1*travel_time_of_agent
            done (boolean): _
            info (dict): other information needed - don't really need now though
        """
        obs_dict, rew_dict, done, info_dict = {}, {}, {}, {}
        
        
        for agent, action in action_dict.items():
            # Create action dictionary
            action_dict = {}
            for i in range(len(action)):
                action_dict[self.routes[i]] = action[i]
            
            # Calculate the travel times and store flow distributions and travel times (Edit for Multi-agent)
            travel_times_dict = self.network.calculate_ttime(action_dict)
            self.__travel_times.append(travel_times_dict)
            
            #Transform dictionary into list
            travel_times = []
            for route in self.routes:
                travel_times.append(travel_times_dict[route])
            
            # Calculate the reward for the population - mean (negative) travel time (Edit for MA w/ different routes)
            reward = np.dot(np.array(action), 
                            -1*np.array(travel_times))
            
            obs_dict[agent] = np.array(travel_times) 
            rew_dict[agent] = reward
            done[agent] = True
            info_dict[agent] = {}
            
            # Add data to the storage bins
            self.add_data(action, travel_times, reward)
                            
        done["__all__"] = True
        return obs_dict, rew_dict, done, info_dict
    
    def reset(self):
        """Resets the state of the environment and returns an initial observation.
        
        For the initial observation: Make an array of 3 elements corresponding to 3 paths in
        the Braess network. It should have the format ---
        
        state = [<traveltime_ABD>, <traveltime_ACD>, <traveltime_ABCD>] = [2, 2, 0.25]
        """
        # Calculate initial travel times with 0 flow on the network
        flows = {route: 0 for route in self.routes}
        initial_dict = self.network.calculate_ttime(flows)
        
        # Turn initial observation to an array
        t_0 = []
        for route in self.routes:
            t_0.append(initial_dict[route])
        
        # State will be a numpy array
        self.state = {'population_1': np.array(t_0)}
        
        # Reset the data bins and insert first item
        if os.path.exists(filename):
            os.remove(filename)
        self.file = open(filename, "a+")
        self.file.write(str(t_0) + "\n")
        self.file.close()
        
        
        return self.state
    
    def add_data(self, path_flow, travel_time, reward):
        self.file = open(filename, "a")
        self.file.write(str(path_flow) + ';' + str(travel_time) + ';' + str(reward) + '\n')
        self.file.close()
        return

In [22]:
class BraessNetwork(object):
    """Stores the cost for all links. Handles calculating the cost of a path given action
       of every car.
    """
    def __init__(self, params=None):
        self.__links = {
            "AB": lambda f: 1 + (f/100),
            "AC": lambda _: 2,
            "BD": lambda _: 2,
            "CD": lambda f: 1 + (f/100),
            "BC": lambda _: 0.25
        } # Dictionary of links and their congestion functions
        self.__paths = {
            "ABD": ("AB", "BD"),
            "ACD": ("AC", "CD"),
            "ABCD": ("AB", "BC", "CD")
        } # Dictionaries of paths to links
        self.total_flow = 100  # 100 cars in total on this network
        return 
    
    @property
    def routes(self):
        """Gives a list of all possible paths in the network to the environment. 
           The environment could then assign an action number to each path. 
        """
        return ("ABD", "ACD", "ABCD")
    
    def calculate_ttime(self, flows):
        """Given a dictionary of paths and flows, this function returns a dictionary of 
           paths and travel time (secs), a.k.a ttime.
           
           Arg:
               flows (dictionary): A dictionare where the key correspond to a path in the network of one o-d pair
                                   and the value corresponds to the flow on that path. Flow will be a float
                                   between 0 and 1 represent the percent of flow. 
           
           Returns: 
               travel_times (list): A list of travel times, order matters 
                                    --> according to the order in my list of paths.
        """
        congestion = {}
        for path in flows:
            links = self.__paths[path]
            for link in links:
                if link not in congestion:
                    congestion[link] = 0
                congestion[link] += flows[path] * self.total_flow
        
        t_time = {}
        for path in flows:
            total_time = 0
            # Calculate travel time of path by adding the congestion time of every 
            # link in that path
            links = self.__paths[path]
            for link in links:
                t_time_func = self.__links[link]
                total_time += t_time_func(congestion[link])
            t_time[path] = total_time
        
        
        return t_time

In [23]:
class Network2(object):
    """Stores the cost for all links. Handles calculating the cost of a path given action
       of every car.
    """
    def __init__(self):
        self.__links = {
            "01": lambda f: f + 2., 
            "04": lambda f: f/2,
            "05": lambda f: f,
            "51": lambda f: f/3,
            "45": lambda f: 3*f, 
            "43": lambda f: f, 
            "24": lambda _: 0.5,
            "23": lambda f: f + 1.,
            "53": lambda f: f/4
        } # Dictionary of links and their congestion functions
        self.__paths = {
            "01": ("01","11"),
            "051": ("05", "51"),
            "0451": ("04", "45", "51"),
            "23": ("23","33"),
            "243": ("24","43"),
            "2453": ("24","45","53")
        } # Dictionaries of paths to links
        return 
    
    def paths(self, population):
        """Gives a list of all possible paths in the network to the environment. 
           The environment could then assign an action number to each path. 
        """
        if population == 0:
            return ("01", "051", "0451")
        elif population == 1:
            return ("23", "243", "2453")
        else:
            return "no such population"
        
    def shared_link(self): # a simple link for this example, need more generalized utility function for more comlicated networks
        return "45" 
    
    def calculate_ttime(self, flows): # flows now is a dictionary; add flow before feeding into the cost fct
        """Given a dictionary of paths and flows, this function returns a dictionary of 
           paths and travel time (secs), a.k.a ttime.
           
           Returns: 
               travel_times (dictionary): A dictionary of paths to their travel times
        """
        congestion = {}
        for population in flows:
            for path in flows[str(population)]:
                links = self.__paths[path]
                for link in links:
                    if link not in self.__links:
                        break
                    if link not in congestion:
                        congestion[link] = 0
                    congestion[link] += flows[str(population)][path]
                    
        t_time = {}
        for population in flows:
            t_time[population] = {}
            for path in flows[population]:
                total_time = 0
                # Calculate travel time of path by adding the congestion time of every 
                # link in that path
                links = self.__paths[path]
                for link in links:
                    if link not in self.__links:
                        break
                    t_time_func = self.__links[link]
                    total_time += t_time_func(congestion[link])
                t_time[population][path] = total_time
        
        return t_time
        
        
    def calculate_ttime_lambda(self, flows, Lambda):
        """Given a dictionary of paths and flows, this function returns a dictionary of 
           paths and travel time, considering the social factor lambda (secs).
           
           Returns: 
               travel_times (dictionary): A dictionary of paths to their travel times,
               considering the social factor lambda
        """
        congestion = {}
        for population in flows:
            for path in flows[str(population)]:
                links = self.__paths[path]
                for link in links:
                    if link not in self.__links:
                        break
                    if link not in congestion:
                        congestion[link] = 0
                    congestion[link] += flows[str(population)][path]
        
        t_time_lambda  = {}
        for population in flows:
            t_time_lambda[population] = {}
            for path in flows[population]:
                total_time = 0
                # Calculate travel time of path by adding the congestion time of every 
                # link in that path
                links = self.__paths[path]
                for link in links:
                    if link not in self.__links:
                        break
                    if link == "01" or link == "05" or link == "43" or link == "23":
                        total_time += Lambda * congestion[link]
                    elif link == "04":
                        total_time += Lambda * congestion[link] * 0.5
                    elif link == "51":
                        total_time += Lambda * congestion[link] / 3
                    elif link == "45":
                        total_time += Lambda * congestion[link] * 3
                    elif link == "53":
                        total_time += Lambda * congestion[link] / 4
                    t_time_func = self.__links[link]
                    total_time += t_time_func(congestion[link])
                t_time_lambda [population][path] = total_time
        return t_time_lambda

In [24]:
# Register the environments
# All environments in this project
routing_envs = {"Braess": BraessEnv}

In [29]:
# Experiment Configurations

EXP_NUM = 0
# time horizon of a single rollout
HORIZON = 1
# number of rollouts per training iteration
N_ROLLOUTS = 1
# number of parallel workers
N_CPUS = 2
# number of steps
T = 10000


single_env = "braess"
    
    
    
def register_env(env_name):
    try:
        register(
            id=env_name,
            entry_point="BraessEnv", # Check if this is correct.
            kwargs={
                "env_params": {},
                "network": {},
            })
    except Exception:
        pass

def run_model(rollout_size=1, num_steps=T):
    """Run the model for num_steps if provided. The total rollout length is rollout_size."""
    register_env(single_env)
    env = DummyVecEnv([lambda: gym.envs.make(single_env)])  # The algorithms require a vectorized environment to run

    model = PPO2('MlpPolicy', env, verbose=1, n_steps=rollout_size)
    model.learn(total_timesteps=num_steps)
    return model


In [33]:
class TestRunModel(unittest.TestCase):
    def setUp(self):
        pass
    
    def testEnvsCanBeLoadedViaModules(self):
        
    
    
    def testModelIsWhatIExpectItToBe:
        """This code tests whether the model made from 'run_model' is set up for the iterative game.
        This means:
         - n_steps = 1 # This is the number of environment steps per update
         - gamma = 0.99 - or - gamma_i = 20/(10 + i)
         - nminibatches: # (int) Number of training minibatches per update
         - noptepochs: # (int) Number of epoch when optimizing the surrogate
        """
        pass
    
    def testEnvConstructorCreatesHowIExpectTheEnvToBeMade:
        pass
    
    def testAnEnvIsSuccessfullyRegisteredInGym:
        pass
    
    def testAnyOtherFunctionalityThatsSpecificToStableBaselines:
        pass
    
    def testPipelineWork(self):
        pass

SyntaxError: invalid syntax (<ipython-input-33-591afd95e99d>, line 3)

In [31]:
EXP_NUM = 0
# time horizon of a single rollout
HORIZON = 1
# number of rollouts per training iteration
N_ROLLOUTS = 1
# number of parallel workers
N_CPUS = 1
# number of steps
T = 10000

In [32]:
if __name__ == "__main__":
    # Add this as the last thing to do for stable baseline conversion.

    model = run_model(N_ROLLOUTS, T)
    # Save the model to a desired folder and then delete it to demonstrate loading
#     if not os.path.exists(os.path.realpath(os.path.expanduser('~/baseline_results'))):
#         os.makedirs(os.path.realpath(os.path.expanduser('~/baseline_results')))
#     path = os.path.realpath(os.path.expanduser('~/baseline_results'))
#     save_path = os.path.join(path, args.result_name)

    self.file = open(filename2, "w+")
    self.file.close()
    print('Saving the trained model!')
    model.save(filename2)
    del model

    # Replay the result by loading the model
    print('Loading the trained model and testing it out!')
    model = PPO2.load(filename2)
#     flow_params = get_flow_params(os.path.join(path, args.result_name) + '.json')
#     flow_params['sim'].render = True
    env = DummyVecEnv([lambda: BraessEnv()])  # The algorithms require a vectorized environment to run
    obs = env.reset()
    reward = 0
    # Should I have it run only once or multiple times to test it out. I'll probably want to run it multiple times
    # Currently HORIZON is only 1
    for i in range(HORIZON):
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        reward += rewards
    print('the final reward is {}'.format(reward))

Error: Attempted to look up malformed environment ID: b'braess'. (Currently all IDs must be of the form ^(?:[\w:-]+\/)?([\w:.-]+)-v(\d+)$.)

In [6]:
### 
# MDP: (All characteristics of this MDP is given in W. Krichene's Paper 
#                            -- "Learning Nash Equilibria in Congestion Games")
#
#  - Observations/States: Each player will observe the cost (travel time) on all of the paths (according to paper)
#                         If the player only observes the loss she incurs then it becomes a multiarmed bandit 
#                         setting.
#  - Actions: Each player will choose a path, using a randomized/mixed strategy. 
#             This means we have a stochastic policy.
#  - Reward: The *cost* of each player will be the travel time that they've incurred on their path. T
#            Each player wants to minimize their travel time. For reward, we can maximize the negative cost.
#  - Model of the environment: We don't have one in this case
#
###

# Test 1: Test that reset() returns an initial observation. 
#         The initial observation should be:
#                "ABD": 3
#                "ACD": 3
#                "ABCD": 2.25
class TestBraessEnv(unittest.TestCase):
    def setUp(self):
        configs = {}
        self.env = BraessEnv(configs)
#         Representing 1 flow of .01 taking path "ABD"
#         flow = {"ABD": 0.01,
#                 "ACD": 0,
#                 "ABCD": 0}
        self.action = {"population_1": np.array([0.01, 0, 0])}  
    
#         flows = {
#             "ABD": 0,
#             "ACD": 0,
#             "ABCD": 0.01
#         } # Flow of 0.01 taking "ABCD"
        self.action2 = {"population_1": np.array([0, 0, 0.01])}
        
    def testFormatOfObservationsandRewardsInStepandReset(self):
        initial = self.env.reset()
        items = list(initial.items())
        self.assertEqual(len(items), 1)
        
        agent, obs = items[0]
        
        # Seems like the "preprocessor" in the reset function turns the dictionary into an numpy array
        # Just check if it has 3 values associated with 3 paths
        self.assertEqual(len(obs), 3)
        
        next_obs, reward, done, info = self.env.step(self.action)
        
        obs_items = list(next_obs.items())
        reward_items = list(reward.items())
        self.assertEqual(len(obs_items), 1)
        self.assertEqual(len(reward_items), 1)
        
        actual_obs = obs_items[0][1]
        actual_reward = reward_items[0][1]
        
        self.assertEqual(len(actual_obs), 3)
        self.assertTrue(isinstance(actual_reward, float))
        self.assertTrue(done["__all__"])
        
        
    def testResetReturnsObservation(self):
        initial = self.env.reset()       
        obs = list(initial.values())[0]
        
#         dict_expected = {
#             "ABD": 3,
#             "ACD": 3,
#             "ABCD": 2.25
#         }
        actual_expected = np.array([3, 3, 2.25])
        np.testing.assert_array_equal(obs, actual_expected)
        
    def testStepReturnsCorrectInformation(self):
        # Test that step returns correct next observations, reward, and termination signal
        env = self.env
        env.reset()

        next_obs, reward, done, _ = env.step(self.action)
        # dict_obs = {
        #     "ABD": 3.01,
        #     "ACD": 3,
        #     "ABCD": 2.26
        # }
        actual_expected_obs = [3.01, 3, 2.26]
        
        # travel_times = {
        #     "ABD": 3.01,
        #     "ACD": 3,
        #     "ABCD": 2.26
        # }
        flow_dist = np.array([0.01, 0, 0])
        path_rewards = np.array([-3.01, -3, -2.26]) # Rewards are negative of travel times
        expected_reward = np.dot(flow_dist, path_rewards)
        
        np.testing.assert_array_equal(actual_expected_obs, list(next_obs.values())[0])
        self.assertEqual(expected_reward, list(reward.values())[0])
        self.assertTrue(done["__all__"])
        
    def testStepSavesNoPrevInfo(self):
        env = self.env
        env.reset()
        env.step(self.action)
    
        next_obs, reward, done, _ = env.step(self.action2)
        
#         dict_obs = {
#             "ABD": 3.01,
#             "ACD": 3.01,
#             "ABCD": 2.27
#         }
#         travel_times = {
#             "ABD": 3.01,
#             "ACD": 3.01,
#             "ABCD": 2.27
#         }
        
        expected_obs = np.array([3.01, 3.01, 2.27])
        
        flow_dist = np.array([0, 0, 0.01])
        path_rewards = np.array([-3.01, -3.01, -2.27])
        expected_reward = np.dot(flow_dist, path_rewards)
        
        np.testing.assert_array_equal(expected_obs, list(next_obs.values())[0])
        self.assertEqual(expected_reward, list(reward.values())[0])
        self.assertTrue(done["__all__"])
        
    def testEnvDoesNotKeepActionsFromPreviousRun(self):
#         env = self.env
#         # Episode 1
#         env.reset()
#         env.step(self.action)
        
#         # Episode 2
#         env.reset()
# #         flows = {
# #             "ABD": 0,
# #             "ACD": 0,
# #             "ABCD": 0.01
# #         } # Flow of 0.01 taking "ABCD"
# # .     {"population_1": np.array([0, 0, 0.01])}
    
#         env.step(self.action2)
        
#         # Check
#         self.fail()
        pass

In [7]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.265s

OK


In [8]:
n = BraessNetwork()
{route: 0 for route in n.routes}

{'ABCD': 0, 'ABD': 0, 'ACD': 0}

In [9]:
# Tests
network = BraessNetwork()

# Test 1 for calculate ttime
#
#                  B
#                / | \                   
#             /    |    \
#          A       |       D
#             \    |    /
#                \ | /
#                  C
#
# Out of 100 cars, we will do: 
#     ABD = 25; 
#     ACD = 25; 
#     ABCD = 50 
# 
# The travel time on each path should result as:
#     ABD = 3.75 (units)
#     ACD = 3.75 (units)
#     ABCD = 3.75 (units)
#
flows = {
    "ABD": 0.25,
    "ACD": 0.25,
    "ABCD": 0.50
}
expect1 = {
    "ABD": 3.75,
    "ACD": 3.75,
    "ABCD": 3.75
}
times = network.calculate_ttime(flows)
print("This is what was given: " + str(times))
print("This is what I expect: " + str(expect1))
print("---")

# Test 2 for calculate ttime
# Out of 100 cars, we will do: 
#     ABD = 50; 
#     ACD = 50; 
#     ABCD = 0 
# 
# The travel time on each path should result as:
#     ABD = 3.5 (units)
#     ACD = 3.5 (units)
#     ABCD = 3.25 (units)  - Even though no one's using this path
flows = {
    "ABD": 0.50,
    "ACD": 0.50,
    "ABCD": 0
}
expect2 = {
    "ABD": 3.5,
    "ACD": 3.5,
    "ABCD": 3.25
}
times = network.calculate_ttime(flows)
print("This is what was given: " + str(times))
print("This is what I expect: " + str(expect2))

This is what was given: {'ABCD': 3.75, 'ABD': 3.75, 'ACD': 3.75}
This is what I expect: {'ABCD': 3.75, 'ABD': 3.75, 'ACD': 3.75}
---
This is what was given: {'ABCD': 3.25, 'ABD': 3.5, 'ACD': 3.5}
This is what I expect: {'ABCD': 3.25, 'ABD': 3.5, 'ACD': 3.5}


In [13]:
# Setup to run experiments
gamma = 0.5
single_pop_network = BraessEnv({})
config = {"gamma": gamma}
policy_graphs = {
    'population': (PPOPolicyGraph, single_pop_network.observation_space, single_pop_network.action_space, config)
}

BRAESS_CONFIG = {
    # === Environment ===
    # Discount factor of the MDP
    "gamma": gamma, # Ask Yiling
    # Number of steps after which the episode is forced to terminate. Defaults
    # to `env.spec.max_episode_steps` (if present) for Gym envs.
    "horizon": 1,
    # Calculate rewards but don't reset the environment when the horizon is
    # hit. This allows value estimation and RNN state to span across logical
    # episodes denoted by horizon. This only has an effect if horizon != inf.
    "soft_horizon": True,
    # Don't set 'done' at the end of the episode. Note that you still need to
    # set this if soft_horizon=True, unless your env is actually running
    # forever without returning done=True.
    "no_done_at_end": True,
    # The default learning rate
    "lr": 0.0001, # Ask Yiling

    # === Evaluation ===
    # Evaluate with every `evaluation_interval` training iterations.
    # The evaluation stats will be reported under the "evaluation" metric key.
    # Note that evaluation is currently not parallelized, and that for Ape-X
    # metrics are already only reported for the lowest epsilon workers.
    "evaluation_interval": None,
    # Number of episodes to run per evaluation period.
    "evaluation_num_episodes": 1,

    # === Multiagent ===
    "multiagent": {
        'policy_graphs': policy_graphs,
        # Function mapping agent ids to policy ids.
        "policy_mapping_fn": tune.function(lambda agent_id: 'population')
    },
}

In [14]:
if __name__ == "__main__":
    env_creator_name = 'multi_routing'
    register_env(env_creator_name, lambda config: BraessEnv(config))
    ray.init()
    experiments = {
        'route-DQN': {
            'run': 'PPO',
            'env': 'multi_routing',
            'stop': {
                'training_iteration': 5
            },
            'config': BRAESS_CONFIG
        },
        # put additional experiments to run concurrently here
    }
    
    run_experiments(experiments)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-09-12_01-22-55_2512/logs.
Waiting for redis server at 127.0.0.1:54865 to respond...
Waiting for redis server at 127.0.0.1:26633 to respond...
Starting the Plasma object store with 6.871947672999999 GB memory using /tmp.

View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=4f93041440b64501ac927342e2dadf9302cfc7dbd9ce3e45

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/0 GPUs
Memory usage on this node: 12.1/17.2 GB

Created LogSyncer for /Users/mtgibson/ray_results/route-DQN/PPO_multi_routing_0_2019-09-12_01-22-55t47ksgcr -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/8 CPUs, 0/0 GPUs
Memory usage on this node: 12.1/17.2 GB
Result logdir: /Users/mtgibson/ray_results/route-DQN
RUNNING trials:
 - PPO_multi_routing_0:	RUNNING

Error processing event.
Traceback (most recent call last):
  File "/anaconda3/envs/marl_routing/lib/python3.5/site-packag

TuneError: ('Trials did not complete', [PPO_multi_routing_0])

# Plots 

The following code plots the RL results.

In [ ]:
import ast

file = open(filename, 'r')
j = 0
# we want to plot the evolution of the flow distribution, of the travel time, and of the reward/average travel time
Actions_plot = np.array([[0, 0, 0]]) # flow path
Reward_plot = np.array([0]) # average travel time/cost or reward
Travel_time_plot = np.array([[0, 0, 0]]) # travel time on each path
while(True):
    j = j+1
    data = file.readline()
#     if j%100 != 1:
#         continue
    try:
#         action_dict = ast.literal_eval("{" + actions.split('{')[1].split('}')[0]+ "}")
#         reward_dict = ast.literal_eval("{" + rewards.split('{')[1].split('}')[0]+ "}")
        
        data = data.split(';')
        
    
#         network = Networks.network(network_name, nb_veh)
#         travel_time, marginal_cost, rew_dict = get_tt_mc(action_dict, network, 0)
        
#         actions_np = np.fromiter(action_dict.values(), dtype=int)
#         Actions_plot = np.append(Actions_plot, [actions_np], axis=0)
#         rewards_np = np.fromiter(reward_dict.values(), dtype=float)
#         Reward_plot = np.append(Reward_plot, [rewards_np], axis=0)
#         travel_time_np = np.fromiter(travel_time.values(), dtype=float)
#         Travel_time_plot = np.append(Travel_time_plot, [travel_time_np], axis=0)
#         if(j==1):
#             print("------ First iteration ------")
#             print("Path choice: " + str(action_dict))
#             print("Reward ray: " + str(reward_dict))
#             print("Travel time paths: " + str({"path " + str(i): network.travel_time(i) for i in range(3)}))
#             print("Travel time cars: " + str(travel_time))
#             print("Marginal cost: " + str(marginal_cost))
#             print("Reward network: " + str(rew_dict))
    except:
        print()
        print("------ Last iteration ------")
        print("Path choice: " + str(action_dict))
        print("Reward ray: " + str(reward_dict))
        print("Travel time paths: " + str({"path " + str(i): network.travel_time(i) for i in range(3)}))
        print("Travel time cars: " + str(travel_time))
        print("Marginal cost: " + str(marginal_cost))
        print("Reward network: " + str(rew_dict))
        break